<a href="https://colab.research.google.com/github/destiny-park/DataAnalysis/blob/main/BidData_sample.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

빅데이터 분석기사 작업용 데이터가 들어있는 Google Drive 마운트


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


## **작업형 1번**
mtcars.csv 데이터의 qsec 컬럼을 최소최대 척도(Min-Max Scale)로 변환한 후 0.5 보다 큰 값을 가지는 레코드의 수를 구하시오

In [ ]:
import pandas as pd
from sklearn.preprocessing import MinMaxScaler


mtcars = r'/content/drive/MyDrive/DataAnalysis/BigData_sample/mtcars.csv'
df = pd.read_csv(mtcars)

mm = MinMaxScaler()
mm.fit(df['qsec'].values.reshape(-1,1))

df['scaling'] = mm.transform(df['qsec'].values.reshape(-1,1))
df_new = df.loc[df['scaling']>0.5]


print(df_new.shape[0])


9


## **작업형 2번**

**제공데이터 목록 :**
1) y_train.csv : 고객의 성별 데이터 (학습용), CSV 파일의 형식
2) X_train.csv, X_test.csv : 고객의 상품구매 속성 (학습용 및 평가용), CSV 형식의 파일

**데이터 형식 및 내용**
1) y_train : 3500 명데이터
-cust_id : 고객의 ID
_gender : 고객의 성별 (0:여자, 1:남자)

2) X_train.csv (3500명 데이터), X_test (2482명 데이터)

**문제**
고객 350명에 대한 학습용 데이터(y_train.csv, X_train.csv)를 이용하여 성별예측 모형을 만든 후, 이를 평가용 데이터(X_test.csv)에 적용하여 얻은 2,482명 고객 성별 예측값(남자일 확률)을 다음과 같은 형식의 CSV 파일로 생성하시오(제출한 모델의 성능은 ROC-AUC 평가 지표에 따라 채점)

**<유의사항>**
성능이 우수한 예측모형을 구축하기 위해서는 적절한 데이터 전처리, Feature Engineering, 분류 알고리즘 사용, 초매개변수 최적화, 모형 앙상블 등이 수반되어야 한다.


**작업 순서**

1.   라이브러리 불러오기
2.   데이터불러오기
3. EDA수행
4. 데이터 전처리(MED)
5. 피처엔지니어링 (sklearn.preprocessing)
6. 모델링 & 하이퍼 파라미터 튜닝 & 앙상블
7. CSV 파일 내보내기


In [ ]:
#1.라이브러리
import pandas as pd

#2.데이터불러오기
test = pd.read_csv("/content/drive/MyDrive/DataAnalysis/BigData_sample/X_test.csv",encoding='euc-kr')
X = pd.read_csv("/content/drive/MyDrive/DataAnalysis/BigData_sample/X_train.csv",encoding='euc-kr')
y = pd.read_csv("/content/drive/MyDrive/DataAnalysis/BigData_sample/y_train.csv",encoding='euc-kr')

print("Reading dataset completed")


Reading dataset completed


In [ ]:
#3.EDA
#--> HIDI : head(), info(), describe(), isnull() 수행을 통한 데이터 탐색

#3-1) 데이터전처리 
#--> MED : M-결측값처리_fillna(), E-인코딩_get_dummies(), D-Drop
#결측값 제거
X = X.fillna(0)
test = test.fillna(0)

#ID값은 학습 시 필요하지 않으므로 삭제
X = X.drop(['cust_id'],axis=1)
cust_id = test.pop('cust_id') #CSV 파일 생성시 필요하므로, 잠시 넣어둠

In [ ]:
#3-2) 피처엔지니어링
# 범주형 필드에 대해 레이블 인코딩 (수치화 하는 작업)
from sklearn.preprocessing import LabelEncoder
cols = ['주구매상품','주구매지점']
for col in cols :
  le = LabelEncoder()
  X[col] = le.fit_transform(X[col])
  test[col] = le.fit_transform(test[col])
 

In [ ]:
#4. 모델링 & 하이퍼 파라미터 튜닝 & 앙상블
from sklearn.ensemble import RandomForestClassifier
model = RandomForestClassifier(n_estimators=100, max_depth=5, random_state=2022)
model.fit(X,y['gender'])
print(model.score(X,y['gender']))
predictions = model.predict_proba(test)



0.6874285714285714


In [ ]:
#5. csv 내보내기
output = pd.DataFrame({'cust_id':cust_id,"gender":predictions[:,1]})
output.to_csv("/content/drive/MyDrive/DataAnalysis/BigData_sample/0000.csv",index=False)